In [123]:
from torchvision.datasets import MNIST
import torchvision.transforms as transforms

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

train_dataset = MNIST('data', train=True, transform=train_transform, download=True)
test_dataset = MNIST('data', train=False, transform=transforms.ToTensor(), download=True)

In [124]:
import torch
from torch import optim
from torch import nn
from torch.utils.data import DataLoader

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, dropout= 0.5):
        super(NeuralNetwork, self).__init__()

        self.w1 = nn.Linear(input_size,hidden_size)
        self.w2 = nn.Linear(hidden_size,out_size)
        self.dropout1 = nn.Dropout(p=dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.w1(x)
        x = self.sigmoid(x)
        x = self.dropout1(x)
        x = self.w2(x)
        return x

In [145]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = 784
hidden_size = 100
out_size = 10
learning_rate = 0.001
batch_size = 64
epochs = 50
dropout = 0.25

In [146]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size, shuffle=False)

In [ ]:
model = NeuralNetwork(input_size, hidden_size, out_size, dropout).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device)
        targets = targets.to(device)

        data = data.reshape(data.shape[0], -1)

        scores = model(data)
        loss = criterion(scores,targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}")

In [ ]:
prediction_list = []

def check_accuracy(loader, model, test=False):
    global prediction_list
    
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            x = x.reshape(x.shape[0], -1)
            
            scores = model(x)
            _, predictions = scores.max(1)
            
            if test:
                prediction_list.extend(predictions.numpy())


            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    
    model.train()
    return num_correct / num_samples

In [150]:
print(f"Accuracy at training: {check_accuracy(train_loader,model)*100:.4}%")
print(f"Accuracy at testing: {check_accuracy(test_loader,model,True)*100:.4}%")

Accuracy at training: 98.85%
Accuracy at testing: 98.23%


In [151]:
#output in csv
import pandas as pd

data = {
    "ID": [],
    "target": [],
}

for i, label in enumerate(prediction_list):
    data["ID"].append(i)
    data["target"].append(label)

df = pd.DataFrame(data)
df.to_csv("submission.csv", index=False)